In [5]:
import os
import pandas as pd
import json

In [6]:
json_dir = 'json'
csv_dir = '../csv'

if not os.path.isdir(csv_dir):
    raise NameError('csv folder doesnt exist')

os.makedirs(json_dir, exist_ok=True)

In [7]:
def get_csv_data(file_name, columns_title):
    file_path = os.path.join(csv_dir, f'{file_name}.csv')
    data = pd.read_csv(file_path, index_col=0)
    data = data.unstack().to_frame()
    data.columns = [columns_title]
    return data

In [21]:
cases = get_csv_data('total_cases', 'cases')
new_cases = get_csv_data('new_cases', 'new_cases')
cases = cases.join(new_cases)

In [16]:
cases

cases  new_cases
         date                        
World    2019-12-31   27.0       27.0
         2020-01-01   27.0        0.0
         2020-01-02   27.0        0.0
         2020-01-03   44.0       17.0
         2020-01-04   44.0        0.0
...                    ...        ...
Zimbabwe 2020-03-24    2.0        0.0
         2020-03-25    2.0        0.0
         2020-03-26    3.0        1.0
         2020-03-27    3.0        0.0
         2020-03-28    5.0        2.0

[17533 rows x 2 columns]

In [22]:
cases['percentage'] = cases.pct_change(axis=1,fill_method='ffill')['new_cases']

In [23]:
cases

cases  new_cases  percentage
         date                                    
World    2019-12-31   27.0       27.0    0.000000
         2020-01-01   27.0        0.0   -1.000000
         2020-01-02   27.0        0.0   -1.000000
         2020-01-03   44.0       17.0   -0.613636
         2020-01-04   44.0        0.0   -1.000000
...                    ...        ...         ...
Zimbabwe 2020-03-24    2.0        0.0   -1.000000
         2020-03-25    2.0        0.0   -1.000000
         2020-03-26    3.0        1.0   -0.666667
         2020-03-27    3.0        0.0   -1.000000
         2020-03-28    5.0        2.0   -0.600000

[17533 rows x 3 columns]

In [26]:


deaths = get_csv_data('total_deaths', 'deaths')

new_deaths = get_csv_data('new_deaths', 'new_deaths')

df = cases.join(deaths).join(new_deaths)
df = df.reset_index(level=[1])

In [28]:
data = {}
for group in df.groupby(level=0):
    data[group[0]] = { 
        'cases': group[1][['date', 'cases']].values.tolist(),
        'deaths': group[1][['date', 'deaths']].values.tolist(),
        'newCases': group[1][['date', 'new_cases']].values.tolist(),
        'newCasesPerc': group[1][['date', 'percentage']].values.tolist(),
        'newDeaths': group[1][['date', 'new_deaths']].values.tolist(),
    }

In [29]:
file_name = 'full_data'
open(f'./json/{file_name}.json', 'w').write(json.dumps(data))

1906929